In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from keras import layers
from keras import models
from keras.datasets import mnist
from keras.utils import to_categorical
import os
import shutil

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Tensorflow로 CNN 구현

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

W0819 02:21:19.064249 139992178198400 deprecation.py:323] From <ipython-input-3-6fa84048fdd1>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0819 02:21:19.069910 139992178198400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0819 02:21:19.073331 139992178198400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0819 02:21:19.844256 139992178198400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0819 02:21:19.847492 139992178198400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz


W0819 02:21:20.165961 139992178198400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [0]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])      # [샘플수, 높이, 너비, 채널 (흑백이라 1채널)]
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# 순서: conv2d -> relu -> pooling (옵션)
w1 = tf.Variable(tf.random_normal([3, 3, 1, 32]))    # [필터 높이, 필터 너비, 채널 특성 수(흑백이니까 1), 필터 갯수]    # 첫번째 계층의 weight
L1 = tf.nn.conv2d(x, w1, strides=[1, 1, 1, 1], padding="SAME")    # 한칸씩 이동하는 convolutional layer    # strides=[1, 칸, 줄 1] -- 양쪽 끝의 1은 고정이라고 생각하고 중간 왼쪽 1은 왼쪽에서 오른쪽으로 1칸 이동, 중간 오른쪽 1은 위에서 아래로 1칸 이동한다는 뜻
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")    # kernel size도 strides와 마찬가지로 양 끝은 1로 고정이고 중간이 사이즈 -- 사이즈는 설계하기 나름인데 사이즈가 작을수록 성능은 좋지만 비용이 크다
# x의 shape: [None, 28, 28, 1]
# w1의 shape: [3, 3, 1, 32]
# L1의 shape - CONV 후: [None, 28, 28, 32]    # padding이 "SAME"이므로 크기는 안바뀌는데, 필터가 32개 추가되므로 채널은 32개로 늘어난다
# L1의 shape - pooling 후: [None, 14, 14, 32]    # 2x2 를 1개로 줄이므로 가로 세로 모두 1/2이 됨

w2 = tf.Variable(tf.random_normal([3, 3, 32, 64]))    # 채널의 갯수가 32로 바뀐 이유는 위 계층에서 필터를 32개 주었으므로 이제는 채널이 32개이다    # 필터 갯수는 내맘대로 64개로 줌
L2 = tf.nn.conv2d(L1, w2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
# L2의 shape - CONV 후: [None, 14, 14, 64]
# L2의 shape - pooling 후: [None, 7, 7, 64]

# 이번엔 fully-connected layer: 입력 7*7*64 -> 출력 256
w3 = tf.Variable(tf.random_normal([7*7*64, 256]))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, w3)    # [-1, 256]
L3 = tf.nn.relu(L3)
# dropout
L3 = tf.nn.dropout(L3, keep_prob)

w4 = tf.Variable(tf.random_normal([256, 10]))
model = tf.matmul(L3, w4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [30]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
for epoch in range(15):
  total_cost = 0
  for i in range(total_batch):    # total_batch = 600
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs.reshape(-1, 28, 28, 1)
    _, cv = sess.run([optimizer, cost], feed_dict={x:batch_xs.reshape(-1, 28, 28, 1), y:batch_ys, keep_prob:0.7})    # keep_prob: 몇% 학습시킬거냐
    total_cost += cv
  print("epoch: ", "%d" % (epoch + 1), ", 평균 cost: ", "{:.3f}".format(total_cost/total_batch))
print("모델 생성됨")

epoch:  1 , 평균 cost:  300.328
epoch:  2 , 평균 cost:  2.221
epoch:  3 , 평균 cost:  1.930
epoch:  4 , 평균 cost:  1.818
epoch:  5 , 평균 cost:  1.889
epoch:  6 , 평균 cost:  1.827
epoch:  7 , 평균 cost:  1.950
epoch:  8 , 평균 cost:  2.031
epoch:  9 , 평균 cost:  2.077
epoch:  10 , 평균 cost:  2.120
epoch:  11 , 평균 cost:  2.125
epoch:  12 , 평균 cost:  2.235
epoch:  13 , 평균 cost:  2.216
epoch:  14 , 평균 cost:  2.296
epoch:  15 , 평균 cost:  2.302
모델 생성됨


In [31]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))    # TRUE or FALSE
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
acc = sess.run(accuracy, feed_dict={x:mnist.test.images.reshape(-1, 28, 28, 1), y:mnist.test.labels, keep_prob:1})
print("정확도: ", acc)

정확도:  0.0981


# Tensorflow의 layers 서브패키지를 활용한 CNN 모델링

In [0]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# tf.layers.conv2d(입력 데이터, 필터 갯수, 필터 크기, 활성화함수)
L1 = tf.layers.conv2d(x, 32, [3, 3], activation=tf.nn.relu)
# tf.layers.max_pooling2d(inputs, pool size, strides)
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7)

L2 = tf.layers.conv2d(L1, 64, [3, 3], activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L1 = tf.layers.dropout(L3, 0.5)

model = tf.layers.dense(L3, 10, activation=None)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
train = tf.train.AdamOptimizer(0.001).minimize(cost)

In [44]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
for epoch in range(15):
  total_cost = 0
  for i in range(total_batch):    # total_batch = 600
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs.reshape(-1, 28, 28, 1)
    _, cv = sess.run([train, cost], feed_dict={x:batch_xs.reshape(-1, 28, 28, 1), y:batch_ys, keep_prob:0.7})    # keep_prob: 몇% 학습시킬거냐
    total_cost += cv
  print("epoch: ", "%d" % (epoch + 1), ", 평균 cost: ", "{:.3f}".format(total_cost/total_batch))
print("모델 생성됨")

epoch:  1 , 평균 cost:  0.173
epoch:  2 , 평균 cost:  0.050
epoch:  3 , 평균 cost:  0.033
epoch:  4 , 평균 cost:  0.025
epoch:  5 , 평균 cost:  0.018
epoch:  6 , 평균 cost:  0.015
epoch:  7 , 평균 cost:  0.013
epoch:  8 , 평균 cost:  0.010
epoch:  9 , 평균 cost:  0.008
epoch:  10 , 평균 cost:  0.007
epoch:  11 , 평균 cost:  0.007
epoch:  12 , 평균 cost:  0.005
epoch:  13 , 평균 cost:  0.006
epoch:  14 , 평균 cost:  0.006
epoch:  15 , 평균 cost:  0.004
모델 생성됨


In [46]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))    # TRUE or FALSE
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
acc = sess.run(accuracy, feed_dict={x:mnist.test.images.reshape(-1, 28, 28, 1), y:mnist.test.labels, keep_prob:1})
print("정확도: ", acc)

정확도:  0.9915


# keras에서 mnist CNN 모델링

In [45]:
from keras import layers
from keras import models

Using TensorFlow backend.


In [0]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))    # (2, 2)의 크기로 max pooling

model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation="relu"))

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.add(layers.Flatten())

In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.add(layers.Dense(64, activation="relu"))

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
Total para

In [0]:
model.add(layers.Dense(10, activation="softmax"))

In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [56]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [57]:
train_images.shape

(60000, 28, 28)

In [0]:
train_images = train_images.reshape((60000, 28, 28, 1))    # 차원을 한 차원 늘려서 채널의 갯수 정보 추가
train_images = train_images.astype("float32")/255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32")/255

In [59]:
test_labels.shape

(10000,)

In [0]:
# 원핫인코딩
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [0]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [64]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

W0819 06:07:15.973359 139992178198400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
60000/60000 [==============================] - 52s 875us/step - loss: 0.1694 - acc: 0.9476
Epoch 2/5
60000/60000 [==============================] - 52s 868us/step - loss: 0.0483 - acc: 0.9852
Epoch 3/5
60000/60000 [==============================] - 52s 863us/step - loss: 0.0329 - acc: 0.9900
Epoch 4/5
60000/60000 [==============================] - 52s 863us/step - loss: 0.0246 - acc: 0.9923
Epoch 5/5
60000/60000 [==============================] - 52s 874us/step - loss: 0.0190 - acc: 0.9942


In [65]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 3s 323us/step


0.9915

# Dogs vs. Cats 데이터셋 관리 (작게 나누기)

In [0]:
import os
import shutil

In [0]:
# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_dir = '/content/drive/My Drive/190000 멀티캠퍼스/Data/dogs-vs-cats/train'

In [0]:
# 소규모 데이터셋을 저장할 디렉터리
base_dir = '/content/drive/My Drive/190000 멀티캠퍼스/Data/dogs-vs-cats/cats_and_dogs_small'
if os.path.exists(base_dir):  # 반복적인 실행을 위해 디렉토리를 삭제합니다.
    shutil.rmtree(base_dir)   # 이 코드는 책에 포함되어 있지 않습니다.
os.mkdir(base_dir)

In [0]:
# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In [0]:
# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [0]:
# 처음 1,000개의 고양이 이미지를 train_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# 다음 500개 고양이 이미지를 validation_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 고양이 이미지를 test_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

In [0]:
# 처음 1,000개의 강아지 이미지를 train_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 validation_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 test_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)